# TP : RL OpenAI Gym

## Objectifs du TP

- Découvrir les propriétés de l'environnement du problème du Taxi sur Gym
- Implémenter l’algorithme du Q-Learning
- Essayer différents paramètres pour l’équation de Bellman
- Exécuter l’algorithme du Q-Learning sur un autre environnement : MountainCar

## Python et importation des bibliothèques utilisées pour le TP

Pour ce TP, Python 3 est utilisé ainsi que plusieurs bibliothèques qu'il faut charger :
1. **gym** : permet de développer et de comparer des algorithmes d'apprentissage par renforcement.
- Documentation : https://gym.openai.com/docs/
- Tous les environnements : https://gym.openai.com/envs/#classic_control
1. **numpy** : permet de manipuler efficacement des grandes matrices  
http://www.numpy.org
1. **matplotlib** : permet de tracer et visualiser des données sous formes de graphiques
https://matplotlib.org

### Si vous utilisez un PC du département, lancez le notebook avec Jupyter (commande : jupyter notebook) et installez les modules suivants.

In [1]:
!pip install gym
!pip install matplotlib
!pip install pyglet

You should consider upgrading via the 'c:\python\python38\python.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'c:\python\python38\python.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'c:\python\python38\python.exe -m pip install --upgrade pip' command.


### **Important** : Pour éviter de faire planter le notebook, ne pas oublier de faire un env.close() à la fin de chaque cellule où vous affichez l'environnement avec env.render()</font>

In [1]:
# Import des modules nécessaires
import gym
import numpy as np
import matplotlib.pyplot as plt
import time
from IPython.display import clear_output

## **A. Taxi**

Commencons par reprendre l'exemple du Taxi vu en cours pour découvrir Gym et implémenter l'algorithme du Q-Learning.

A titre d'information, voici le code source de l'environnement : https://github.com/openai/gym/blob/master/gym/envs/toy_text/taxi.py

In [2]:
# Configuration de l'environnement
env = gym.make('Taxi-v3')

### **1. Découverte de l'environnement**

Dans un premier temps, l'objectif va être de se familiariser avec les attributs et méthodes de Gym pour retrouver les propriétés de l'environnement Taxi-v3 énoncées dans le cours. Ces informations permettront de faciliter par la suite la compréhension et l'implémentation de l'algorithme du Q-Learning.

Pour obtenir une description de l'environnement, utiliser **help(env.unwrapped)**

<font color=orange>**Question 1** : Afficher l'environnement de Taxi.</font>

In [3]:
env.reset()
env.render()

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+



<font color=orange>**Question 2** : Quelles récompenses l'agent peut-il obtenir pour une action ? Quelle est la récompense totale minimale à la fin d'une partie ? maximale si l'agent doit juste déposer le passager ?</font>

Pour information, une partie se termine au bout de 200 étapes si le passager n'a pas été déposé.

Max : 20  
  
Min : -2000  
Mouvement illégal à chaque action avec un maximum de 200 steps avant que la partie soit terminée

<font color=orange>**Question 3** : Retrouver les informations sur l'espace d'action présentées en cours (discret, 6 actions et associer à chaque valeur l'action réelle associée). </font>

In [4]:
# Explore the environment
print('Action space:', env.action_space)
# Discrete(3)

#  |  There are 6 discrete deterministic actions:
#  |  - 0: move south
#  |  - 1: move north
#  |  - 2: move east 
#  |  - 3: move west 
#  |  - 4: pickup passenger
#  |  - 5: dropoff passenger

Action space: Discrete(6)


<font color=orange>**Question 4** : Retrouver les informations sur l'espace d'observation présentées en cours (discret, 500 états).</font>

Pour rappel, il y a 500 états car :
- le taxi peut se retrouver à 25 positions différentes (grille de taille 5x5)
- 4 positions (R,G,Y,B) indiquant les positions possibles de départ et d'arrivée du passager
- 5 positions indiquant les positions possibles du passager avec 4 positions pour le départ et l'arrivée (R,G,B,Y) et une position quand le passager est dans le taxi

In [5]:
print('State space:', env.observation_space)
# 25 locations of the taxi (5x5 grid)
# 4 letters corresponding to the locations
# 5 possible passenger locations with the case when the passenger is in the taxi

State space: Discrete(500)


Avant d'implémenter l'algorithme du Q-Learning, nous allons commencer par faire jouer notre agent de manière aléatoire et observer la récompense par partie. Peut-être que sans entraînement, notre agent est capable de prendre puis de déposer le passager à sa destination...

### **2. Agent Random**

<font color=orange>**Question 5** : Générer un agent jouant 2 parties de Taxi de manière random et afficher en sortie la récompense par épisode.</font>  

Exemple de sortie :  
Reward of the episode 0 : -200.0  
Reward of the episode 1 : -200.0   

In [ ]:
# A compléter entièrement

rewards = []

for episode in range(2):
    tot_reward = 0
    
    env.reset() # Indicate something is missing here
    done = False
    while not done:
        clear_output(wait = True)
        env.render()
        time.sleep(0.1)
        observation, reward, done, info = env.step(env.action_space.sample())  # Complete env.action_space.sample()
        tot_reward += reward

    rewards.append(tot_reward)

env.close()

for i in range(len(rewards)):
    print("Reward of the episode", episode, ":", rewards[i])

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (West)
Reward of the episode 1 : -785
Reward of the episode 1 : -668


Il est très probable que votre agent n'arrive jamais à déposer à prendre votre passager et à le déposer à sa destination. Pour y remédier, passons au Q-Learning.

### **3. Q-Learning**

Avec le Q-Learning, votre agent devrait apprendre au fur et à mesure des actions qu'il effectue.

Pour rappel, le Q-Learning permet de mesurer la qualité d'une combinaison état-action en termes de récompenses. Il le fait à l'aide d'une Q-Table qui est mise à jour après chaque action avec sa ligne correspondant à l'état et sa colonne à l'action. Un épisode se termine après la réalisation d'un ensemble d'actions. A la fin de l'entraînement, la Q-Table suggère la politique idéale.  
On dit également du Q-Learning qu'il est un algorithme *off-policy* ce qui signifie que plutôt que de suivre simplement certaines règles de comportement, connues sous le nom de politique, votre agent peut choisir des actions randoms. De plus, le Q-Learning a une approche *model-free* ce qui signifie que l'agent ne peut pas connaître la valeur d'une action avant de la réaliser.
  
La mise à jour de la Q-Table se fait avec l'équation de Bellman : 

![](./images/bellman.png)

Nous allons commencer par définir la Q-Table. Les espaces d'observation et d'action étant discret, celle-ci est de la forme Q(nbEtat, nbAction). Pour l'initialisation de la Q-Table, vous pouvez l'initialiser à 0 ou à tout autre valeur car l'algorithme du Q-Learning garantit la convergence.

<font color=orange>**Question 6** : Définir la Q-Table.</font>

Indice : np.zeros  
  
Exemple : np.zeros((2, 1))  
array([[ 0.],  
       [ 0.]])

In [6]:
Q = np.zeros((env.observation_space.n, env.action_space.n))
Q

array([[0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       ...,
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.]])

<font color=orange>**Question 7** : Compléter les parties manquantes de l'algorithme du Q-Learning.</font>

In [7]:
def QLearning(env, Q, alpha_apprentissage, gamma_actualisation, epsilon, min_eps, episodes):

    # Initialisation de variables pour suivre les rewards
    reward_list = []  
    avg_reward_list = []

    # Calcul d'un facteur de réduction pour epsilon. Pour rappel, epsilon est utilisé lors du choix de l'action pour trouver un compromis entre l'exploration de nouveaux états ou l'exploitation des connaissances déjà acquises.
    # Ce facteur de réduction permet de faire décroître de manière linéaire epsilon au cours du temps afin d'accorder plus de poids aux connaissances acquises dans la Q-Table.
    reduction = (epsilon - min_eps)/episodes

    for episode in range(episodes):

        # Initialisation des paramètres
        tot_reward, reward = 0, 0
        state = env.reset()
        done = False
   
        while done != True:

            # A compléter : afficher l'agent lors des 10 dernières parties de l'entraînement
            if episode >= (episodes - 10):
                clear_output(wait = True)
                env.render()
                time.sleep(0.5)

            # A compléter : déterminer la prochaine action : Epsilon Greedy Policy (trouver un compromis entre exploration et exploitation de la Q-Table)
            if np.random.random() < 1 - epsilon:
                # action = meilleure action de la Q-Table pour l'état en cours
                action = np.argmax(Q[state])
            else:
                # action = random. On découvre autre chose
                action = env.action_space.sample()
                
                
            # Effectuer l'action choisie et récupérer les informations
            state2, reward, done, info = env.step(action)

            # Mettre à jour la Q-Table en utilisant l'équation de Bellman
            Q[state, action] += alpha_apprentissage * (reward + gamma_actualisation * np.max(Q[state2]) - Q[state, action]) 
               
            # Mettre à jour notre récompense totale et l'état
            tot_reward += reward      
            state = state2        
            
        
        # Réduire l'exploration lors du Epsilon Greedy Policy en réduisant l'epsilon pour profiter davantage des connaissances acquises dans la Q-Table.
        # Dans notre cas, on fait réduire epsilon de manière linéaire jusqu'à un minimum spécifié (min_eps) par rapport au nombre total d'épisodes.
        if epsilon > min_eps:
            epsilon -= reduction

        reward_list.append(tot_reward)

        if (episode+1) % 100 == 0:
            avg_reward = np.mean(reward_list)
            avg_reward_list.append(avg_reward)
            reward_list = []
            print('Episode {} Average Reward: {}'.format(episode+1, avg_reward))

    env.close()
        
    return avg_reward_list, Q

<font color=orange>**Question 8** : Exécuter l'algorithme précédent avec la cellule suivante. Un graphique "rewards.jpg" devrait avoir été créé. Celui-ci présente l'évolution de la récompense au fil de l'entrainement. Tester l'algorithme de Q-Learning pour différentes valeurs du facteur d'apprentissage et du facteur d'actualisation.</font>

In [8]:
# Lancer l'apprentissage
rewards, qtable = QLearning(env, Q, alpha_apprentissage=0.2, gamma_actualisation=0.9, epsilon=0.8, min_eps=0.01, episodes=5000)

# Générer le graphique des rewards au cours des épisodes (rewards.jpg)
plt.plot(100*(np.arange(len(rewards)) + 1), rewards)
plt.xlabel('Episodes')
plt.ylabel('Average Reward')
plt.title('Average Reward vs Episodes')
plt.savefig('TaxiRewards.jpg')     
plt.close()  

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (South)
Episode 5000 Average Reward: 6.9


In [18]:
qtable[3]

array([ 0.46029208,  1.62232382,  0.45359832,  1.62256045,  2.9140163 ,
       -7.37746122])

Normalement votre agent est capable de résoudre aisément le problème, félicitations !  
  
A présent, pour montrer que les environnements standardisés sont un atout de Gym, nous allons changer d'environnement et adapter légèrement notre algorithme de Q-Learning à un nouveau problème plus visuel (et un peu plus complexe).

## B. MountainCar

Le problème : 
- Une voiture est placée sur une piste unidimensionnelle entre deux sommets.
- Objectif : gravir le sommet de droite
- Problème : le moteur de la voiture n’est pas assez puissant pour gravir le sommet d’un seul coup

![](./images/mountaincar.jpg)

In [2]:
# Configuration de l'environnement
env = gym.make('MountainCar-v0')

### **1. Découverte de l'environnement**

Dans un premier l'objectif va être de comprendre l'environnement de MountainCar afin de faciliter par la suite la modification de l'implémentation de l'algorithme du Q-Learning.

<font color=orange>**Question 1** : Quand une partie (= épisode) se termine-t-elle d'un point de vue théorique ? (2 cas possibles)</font>

In [3]:
help(env.unwrapped)
#  |  Episode Termination:
#  |       The car position is more than 0.5
#  |       Episode length is greater than 200

Help on MountainCarEnv in module gym.envs.classic_control.mountain_car object:

class MountainCarEnv(gym.core.Env)
 |  MountainCarEnv(goal_velocity=0)
 |  
 |  Description:
 |      The agent (a car) is started at the bottom of a valley. For any given
 |      state the agent may choose to accelerate to the left, right or cease
 |      any acceleration.
 |  
 |  Source:
 |      The environment appeared first in Andrew Moore's PhD Thesis (1990).
 |  
 |  Observation:
 |      Type: Box(2)
 |      Num    Observation               Min            Max
 |      0      Car Position              -1.2           0.6
 |      1      Car Velocity              -0.07          0.07
 |  
 |  Actions:
 |      Type: Discrete(3)
 |      Num    Action
 |      0      Accelerate to the Left
 |      1      Don't accelerate
 |      2      Accelerate to the Right
 |  
 |      Note: This does not affect the amount of velocity affected by the
 |      gravitational pull acting on the car.
 |  
 |  Reward:
 |       Rew

<font color=orange>**Question 2** : Quelle est la récompense totale minimale à la fin d'une partie (pire partie) ? maximale (partie parfaite) ?</font>

Max : 0  
Du premier coup, atteindre le sommet sinon -1 à chaque action  
  
Min : -200  
-1 à chaque action sans atteindre le sommet

<font color=orange>**Question 3** : L'espace des actions de l'environnement est-il discret ou continu ? Décrire les différentes actions (mathématiquement) et l'action réelle associée à chaque valeur.</font>

In [4]:
# Explore the environment
print('Action space:', env.action_space)
# Discrete(3)

# The action space comprises three discrete actions, represented by the integers 0, 1 and 2. (push_left, nothing, push_right)  
# 0 : left  
# 1 : nothing  
# 2 : right  

Action space: Discrete(3)


<font color=orange>**Question 4** : L'espace des observations de l'environnement est-il discret ou continu ? S'il est discret décrire le nombre d'observations et leur signification sinon décrire les bornes inférieures et supérieures. S'il est continu, identifier également à quoi correspondent les composantes des observations.</font>

In [5]:
print('State space:', env.observation_space)
# The state space represents a 2-dimensional box. Each observation is a vector of 2 float values.
print(env.observation_space.low)
print(env.observation_space.high)
# The first element of the state vector, representing the car's position, can take on any value in the range -1.2 to 0.6.
# The second value, representing the car's velocity, can take on any value in the range -0.07 to 0.07.

# To find out what the components of the vector are, use help(env.unwrapped) like before
print(env.observation_space.shape[0])

State space: Box(-1.2000000476837158, 0.6000000238418579, (2,), float32)
[-1.2  -0.07]
[0.6  0.07]
2


<font color=orange>**Question 5** : Dans le cadre du Q-Learning, l'espace d'observation pose problème lors de la création d'une Q-Table, pourquoi ?</font>

L'algorithme du Q-Learning repose sur une Q-table dont la convergence est garantie à condition que chaque paire état-action soit visitée un nombre suffisamment grand de fois. Dans notre cas, nous avons affaire à un espace d'état continu, ce qui signifie qu'il y a une infinité de paires état-action, ce qui rend impossible de satisfaire cette condition et de créer une Q-table de taille finie.  

Pour répondre à ce problème, nous pouvons discrétiser l'espace d'état.

Une autre alternative consiste à utiliser le Deep Q-Learning. Cette méthode consiste à approximer l'algorithme de la Q-table avec un réseau de neurones. Cette solution est pertinente dans le cas de problèmes complexes notamment avec des espaces continus ou lorsque la taille de la Q-Table est très conséquente. Le Deep Q-Learning ne sera pas vu dans le cadre de ce cours mais constitue une bonne ouverture pour ceux voulant en savoir plus, cette solution étant très utilisée. Par ailleurs, pour un problème aussi simple que le MountainCar, on peut s'en passer et obtenir de bons résultats.

Avant d'implémenter l'algorithme du Q-Learning, nous allons commencer par faire jouer notre agent de manière aléatoire et observer la récompense par partie. Peut-être que sans entraînement, notre agent est capable de gravir le sommet...

### **2. Agent Random**

Afin de voir en pratique une première fois l'environnement de MountainCar, réalisons de nouveaun agent random.

<font color=orange>**Question 6** : Générer un agent jouant 3 parties au MountainCar de manière random et afficher en sortie la récompense par épisode.</font>  

Exemple de sortie :  
Reward of the episode 0 : -200.0  
Reward of the episode 1 : -200.0  
Reward of the episode 2 : -200.0  

In [11]:
# A compléter entièrement

for episode in range(3):
    tot_reward = 0
    env.reset() # Indicate something is missing here
    done = False
    while not done:
        env.render()
        observation, reward, done, info = env.step(env.action_space.sample())  # Complete env.action_space.sample()
        tot_reward += reward
    print("Reward of the episode", episode, ":", tot_reward)

env.close()

Reward of the episode 0 : -200.0
Reward of the episode 1 : -200.0
Reward of the episode 2 : -200.0
Reward of the episode 3 : -200.0
Reward of the episode 4 : -200.0
Reward of the episode 5 : -200.0
Reward of the episode 6 : -200.0
Reward of the episode 7 : -200.0
Reward of the episode 8 : -200.0
Reward of the episode 9 : -200.0


Normalement, votre agent ne devrait jamais réussir à remporter une partie. Pour y remédier, passons au Q-Learning.

### **3. Q-Learning**

Cette fois-ci, votre agent devrait apprendre au fur et à mesure des actions qu'il effectue. Attention cependant, l'algorithme de Q-learning converge mais cela peut nécessiter de nombreuses parties d'entraînement.

Pour rappel, le Q-Learning permet de mesurer la qualité d'une combinaison état-action en termes de récompenses. Il le fait à l'aide d'une Q-Table qui est mise à jour après chaque action avec sa ligne correspondant à l'état et sa colonne à l'action. Un épisode se termine après la réalisation d'un ensemble d'actions. A la fin de l'entraînement, la Q-Table suggère la politique idéale.
  
La mise à jour de la Q-Table se fait avec l'équation de Bellman : 

![](./images/bellman.png)

Afin de générer la Q-Table, nous allons devoir discrétiser l'espace d'observation. La Q-Table correspondant à l'ensemble des paires états-actions possibles, nous devons déterminer le nombre d'états possibles pour créer la table aux bonnes dimensions. Notre espace d'observation ayant 2 composantes vectorielles, notre Q-Table sera de la forme Q(s1, s2, a). Comment connaître la taille de s1 et s2 ?  
  
L'espace d'observation ayant pour limite inférieure (min1 = -1.2, min2 = -0.07) et pour limite supérieure (max1 = 0.6, max2 = 0.07), une manière simple de discrétiser consiste à arrondir le premier élément du vector d'état à 0.1 près et le second élément à 0.01.  
Exemple : 
- Pour la première composante dont les limites sont (min = -1.2 et max = 0.6), les valeurs possibles sont les suivantes : -1.2, -1.1, -1.0, ..., 0.5, 0.6. Nous obtenons ainsi N valeurs (à déterminer à la question suivante).

<font color=orange>**Question 7** : Déterminer pour chacune des composantes de l'espace d'observation, le nombre de valeurs possibles après discrétisation.</font>

Indices : env.observation_space.high, env.observation_space.low, np.array([10,100]), np.round, +1  et n'oubliez pas de compter les bornes comme des états.  
En cas de difficultés, vous pouvez déterminer ces 2 valeurs à la main.

In [6]:
# Determine size of discretized state space
print(env.observation_space.high, env.observation_space.low)
num_states = (env.observation_space.high - env.observation_space.low)*np.array([10,100])
print(num_states)
# [18.00000072 14.00000006]
num_states = np.round(num_states, decimals=0).astype(int) + 1
# [19 15]
num_states

# OR

counter = 0
np.arange(-1.2,0.7,0.1).size

[0.6  0.07] [-1.2  -0.07]
[18.00000072 14.00000006]


19

<font color=orange>**Question 8** : Définir la Q-Table (format 3D possible).</font>

In [19]:
# Now, size Q-table : (19x15) x 3 = 855  
# 19 : from -1.2 to 0.6  
# 15 : from -0.07 to 0.07  
# So 19x15 possible pairs  
  
# Q-Learning  
# Not Q(s ,a) but Q(s1, s2, a) because we are dealing with a two-dimensional state space  
  
# Initial state of our environment. Only the velocity is fixed (=0)

Q = np.zeros((num_states[0], num_states[1], env.action_space.n))
Q.shape

(19, 15, 3)

Une fois le nombre de valeurs possibles par composante de l'espace d'observation obtenu, il est nécessaire de faire correspondre les valeurs de l'espace d'observation aux indices de notre table.  
Exemple :   
- Pour la première composante :   
![](./images/mapping_discretization.png)


<font color=orange>**Question 9** : Déterminer pour un état donné sa correspondance dans la Q-Table après discrétisation. Pour essayer votre code, vous pourrez prendre comme état **state = env.reset()**. state est dans ce cas de la forme (X, 0) avec X variable. En sortie la deuxième composante de state après discrétisation devrait être 7.</font>

Exemple :
- state = env.reset() peut retourner [-0.40525229 0.]. La première composante est variable et la seconde est toujours nulle.
- votre script devra retourner [8 7] dans ce cas. Ici -0.405... correespond à l'état 8 de la position. La vitesse (2ème composante) sera toujours 7 car initialement la vitesse est nulle.

Ce morceau de code sera à réutiliser dans l'algorithme du Q-Learning.  
  
Indice : np.array([10, 100]), np.round(..., decimals=0).astype(int)

In [20]:
state = env.reset()
print(state)

state_discretized = (state - env.observation_space.low)*np.array([10, 100])
state_discretized = np.round(state_discretized, 0).astype(int)

print(state_discretized)

[-0.49713689  0.        ]
[7 7]


<font color=orange>**Question 10** : Compléter les parties manquantes de l'algorithme du Q-Learning.</font>

In [21]:
def QLearning(env, Q, alpha_apprentissage, gamma_actualisation, epsilon, min_eps, episodes):

    # Initialisation de variables pour suivre les rewards
    reward_list = []  
    avg_reward_list = []

    # Calcul d'un facteur de réduction pour epsilon. Pour rappel, epsilon est utilisé lors du choix de l'action pour trouver un compromis entre l'exploration de nouveaux états ou l'exploitation des connaissances déjà acquises.
    # Ce facteur de réduction permet de faire décroître de manière linéaire epsilon au cours du temps afin d'accorder plus de poids aux connaissances acquises dans la Q-Table.
    reduction = (epsilon - min_eps)/episodes

    for episode in range(episodes):

        # Initialisation des paramètres
        tot_reward, reward = 0, 0
        state = env.reset()
        done = False

        # A compléter : discrétisation de l'état en cours. Réutiliser la cellule de la question précédente.
        state_discretized = (state - env.observation_space.low)*np.array([10, 100])
        state_discretized = np.round(state_discretized, 0).astype(int)
   
        while done != True:

            # A compléter : afficher l'agent lors des 20 dernières parties de l'entraînement
            if episode >= (episodes - 20):
                env.render()

            # A compléter : déterminer la prochaine action avec Epsilon Greedy Policy (trouver un compromis entre exploration et exploitation de la Q-Table)
            if np.random.random() < 1 - epsilon:
                # action = meilleure action de la Q-Table pour l'état en cours
                action = np.argmax(Q[state_discretized[0], state_discretized[1]])
            else:
                # action = random. On découvre autre chose
                action = env.action_space.sample()
                
                
            # Effectuer l'action choisie et récupérer les informations
            state2, reward, done, info = env.step(action)

            # Discrétiser le nouvel état obtenu
            state2_discretized = (state2 - env.observation_space.low)*np.array([10, 100])
            state2_discretized = np.round(state2_discretized, 0).astype(int)

            # Si l'action fait gagner l'agent, mettre directement à jour la Q-Table avec la récompense
            if done and state2[0] >= 0.5:
                Q[state_discretized[0], state_discretized[1], action] = reward
            else:
                # Mettre à jour la Q-Table en utilisant l'équation de Bellman
                Q[state_discretized[0], state_discretized[1], action] += alpha_apprentissage * (reward + gamma_actualisation * np.max(Q[state2_discretized[0], state2_discretized[1]]) - Q[state_discretized[0], state_discretized[1], action])

            # Mettre à jour notre récompense totale et l'état
            tot_reward += reward      
            state_discretized = state2_discretized        
            
        
        # Réduire l'exploration lors du Epsilon Greedy Policy en réduisant l'epsilon pour profiter davantage des connaissances acquises dans la Q-Table.
        # Dans notre cas, on fait réduire epsilon de manière linéaire jusqu'à un minimum spécifié (min_eps) par rapport au nombre total d'épisodes.
        if epsilon > min_eps:
            epsilon -= reduction

        reward_list.append(tot_reward)

        if (episode+1) % 100 == 0:
            avg_reward = np.mean(reward_list)
            avg_reward_list.append(avg_reward)
            reward_list = []
            print('Episode {} Average Reward: {}'.format(episode+1, avg_reward))

    env.close()
        
    return avg_reward_list, Q

<font color=orange>**Question 11** : Exécuter l'algorithme précédent avec la cellule suivante. Un graphique "rewards.jpg" devrait avoir été créé. Celui-ci présente l'évolution de la récompense au fil de l'entrainement. Tester l'algorithme de Q-Learning pour différentes valeurs du facteur d'apprentissage et du facteur d'actualisation.</font>

Cet apprentissage étant plus long que celui de Taxi, on peut sauvegarder la Q-Table. Ainsi si votre notebook plante ou pour une autre raison vous n'aurez pas à relancer l'apprentissage.

In [23]:
# Lancer l'apprentissage
rewards, qtable = QLearning(env, Q, alpha_apprentissage=0.2, gamma_actualisation=0.9, epsilon=0.8, min_eps=0.01, episodes=5000)
qtable = qtable.reshape(285,3)
np.savetxt("Qtable.csv", qtable, delimiter=";")

Episode 100 Average Reward: -200.0
Episode 200 Average Reward: -200.0
Episode 300 Average Reward: -200.0
Episode 400 Average Reward: -200.0
Episode 500 Average Reward: -200.0
Episode 600 Average Reward: -200.0
Episode 700 Average Reward: -200.0
Episode 800 Average Reward: -200.0
Episode 900 Average Reward: -200.0
Episode 1000 Average Reward: -200.0
Episode 1100 Average Reward: -200.0
Episode 1200 Average Reward: -200.0
Episode 1300 Average Reward: -200.0
Episode 1400 Average Reward: -200.0
Episode 1500 Average Reward: -200.0
Episode 1600 Average Reward: -200.0
Episode 1700 Average Reward: -200.0
Episode 1800 Average Reward: -200.0
Episode 1900 Average Reward: -200.0
Episode 2000 Average Reward: -200.0
Episode 2100 Average Reward: -200.0
Episode 2200 Average Reward: -200.0
Episode 2300 Average Reward: -200.0
Episode 2400 Average Reward: -200.0
Episode 2500 Average Reward: -200.0
Episode 2600 Average Reward: -200.0
Episode 2700 Average Reward: -200.0
Episode 2800 Average Reward: -199.08


In [ ]:
# Générer le graphique des rewards au cours des épisodes (rewards.jpg)
plt.plot(100*(np.arange(len(rewards)) + 1), rewards)
plt.xlabel('Episodes')
plt.ylabel('Average Reward')
plt.title('Average Reward vs Episodes')
plt.savefig('rewards.jpg')     
plt.close()  

Récupération de la Q-Table et restructuration en 3D.

In [ ]:
# Ouverture du fichier Qtable.csv pour récupérer la table apprise puis reshape en 3d
Q = np.loadtxt("Qtable.csv", delimiter=";")
Q = Q.reshape(19,15,3)

<font color=orange>**Question 12** : Utiliser la Q-Table pour résoudre le problème.</font>  

In [ ]:
# Compléter
# Utilisation de la Qtable pour résoudre le problème
done = False
state = env.reset()
state_adj = state*np.array([10, 100])
state_adj = np.round(state_adj, 0).astype(int)


while done != True:
    
    env.render()
    action = np.argmax(Q[state_adj[0], state_adj[1]])
    state, reward, done, info = env.step(action)
    
    state_adj = (state - env.observation_space.low)*np.array([10, 100])
    state_adj = np.round(state_adj, 0).astype(int)

env.close()

### Le TP touche à sa fin... Félicitations !  